# Tutorial: approaches to serverless code writing

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

In [2]:
from typing import List, Union, Optional, Dict

from qiskit import QuantumCircuit, transpile
from qiskit.providers import Backend
from qiskit.circuit.random import random_circuit
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeVigo, FakeAlmaden, FakeBrooklyn, FakeCasablanca

from quantum_serverless import QuantumServerless, distribute_task, get, put

---

There are multiple approaches to write serverless code using ray.

We will review 2 of them:

- exposing all details to user
- hiding details from user

In our examples we will be building functions and classes that can transpile circuits remotely and in parallel manner

### Approach #1: revealing details of implementations to user

One way of writing your modules, classes and functions in a way that user himself handling ray stuff.

Inside your module you will have something like that

In [3]:
# ==================
# Insude your module
# ==================

# your_module.transpiler

@distribute_task()
def remote_transpile(circuits: List[QuantumCircuit], backend: Backend):
    return transpile(circuits=circuits, backend=backend) 

Then the user will use this funciton in following way

In [ ]:
# ================
# User perspective
# ================

# from your_module.transpiler import remote_transpile

serverless = QuantumServerless()

with serverless.context():
    # First we need to put our object to cluster and pass object ids to functions

    # Let's do that for circuits
    circuits = [random_circuit(3, 2) for _ in range(10)]
    circuits_id = put(circuits)

    # and for backends
    backend_ids = []
    for fake_backend in [FakeAlmaden(), FakeBrooklyn(), FakeCasablanca(), FakeVigo()]:
        backend = AerSimulator.from_backend(fake_backend)
        backend_id = put(backend)
        backend_ids.append(backend_id)
    
    # now we need to call remote function and pass all objects to form tasks
    tasks = []
    for backend_id in backend_ids:
        task_id = remote_transpile(circuits_id, backend_id)
        tasks.append(task_id)
        
    print(f"Results: {get(tasks)}")

---

As you can see, the user has to create a lot of boilerplate code and to pass ray objects and pointers across the application.

### Approach #2: hiding details of implementations from user


We can do better by hiding all of these details inside our module. Let's have a look.

In [5]:
# ==================
# Insude your module
# ==================

# your_module.transpiler

@distribute_task()
def remote_transpile(circuits: List[QuantumCircuit], backend: Backend):
    return transpile(circuits=circuits, backend=backend) 


def parallel_transpile(circuits: List[QuantumCircuit], backends: List[Backend]) -> List[QuantumCircuit]:
    circuits = [random_circuit(3, 2) for _ in range(10)]
    circuits_id = put(circuits)

    backend_ids = []
    for backend in backends:
        backend_id = put(backend)
        backend_ids.append(backend_id)
    
    # now we need to call remote function and pass all objects to form tasks
    tasks = []
    for backend_id in backend_ids:
        task_id = remote_transpile(circuits_id, backend_id)
        tasks.append(task_id)
    
    return get(tasks)

Now let's see how code shrinks from the user's perspective, which may, in certain situations, provide a better user experience.

In [6]:
# ================
# User perspective
# ================

# from your_module.transpiler import remote_transpile

circuits = [random_circuit(3, 2) for _ in range(10)]
backends = [AerSimulator.from_backend(backend)
            for backend in [FakeAlmaden(), FakeBrooklyn(), FakeCasablanca(), FakeVigo()]]

serverless = QuantumServerless()

with serverless.context():
    print(f"Results: {parallel_transpile(circuits, backends)}")

Results: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1e0ca0d90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1b0c8bd90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1814c9790>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc181484190>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1e0d243d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc190b5d110>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc190ee9510>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1b0ced110>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1b0ced590>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1814e1150>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1b0c8bcd0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1814e1c10>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc1e0c89150>, <qiskit.circuit.quantumcircuit.Quantum

---

Now we have only 2 lines of code to call our serverless library.
It hides all unnecessary details from the user.